In [7]:
!pip install tensorflow_gan
!pip install tensorflow_hub

     |████████████████████████████████| 368kB 2.7MB/s 


In [0]:
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import matplotlib.pyplot as plt
sys.path.append("../")
import numpy as np
import glob
from tqdm.notebook import tqdm
import time
import tensorflow.compat.v1 as tf
import tensorflow_gan as tfgan
from scipy.linalg import sqrtm
import tensorflow_hub as tfhub

In [31]:
# download official FVD library - see paper 
!wget https://raw.githubusercontent.com/google-research/google-research/master/frechet_video_distance/frechet_video_distance.py
import frechet_video_distance as fvd

--2020-06-05 16:44:05--  https://raw.githubusercontent.com/google-research/google-research/master/frechet_video_distance/frechet_video_distance.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5171 (5.0K) [text/plain]
Saving to: ‘frechet_video_distance.py.1’

frechet_video_dista 100%[===================>]   5.05K  --.-KB/s    in 0s      

2020-06-05 16:44:05 (72.6 MB/s) - ‘frechet_video_distance.py.1’ saved [5171/5171]



In [0]:
# load inception net for the computation of FID and KID
inception = tfhub.load('https://tfhub.dev/tensorflow/tfgan/eval/inception/1')

In [0]:
def get_im_features(x, batch_size = 100):
    x = np.float32(x) / 255
    x = x.reshape(x.shape[0] * x.shape[1], x.shape[2], x.shape[3], x.shape[4])
    nbatch = int(np.ceil(x.shape[0] / batch_size))
    out = []
    for i in tqdm(range(nbatch)):
        b = x[ i*batch_size : (i+1)*batch_size]
        out += inception(b)["pool_3"].numpy(),
    return np.concatenate(out, 0).squeeze()

def get_vid_features(vid):
    # python list to save features
    features = []
    with tf.Graph().as_default():
        videos  = tf.placeholder(tf.float32, [batch_size, VIDEO_LENGTH, 64, 64, 3])
        feature = fvd.create_id3_embedding(fvd.preprocess(videos, (224, 224)))

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            n = vid.shape[0]
            nbatch = int(np.ceil(n / batch_size))
            
            for i in tqdm(range(nbatch), desc="features"):
                idx = slice(batch_size*i,batch_size*(i+1))
                video_batch = vid[idx]
                actual_batch_size = video_batch.shape[0]

                if actual_batch_size < batch_size:
                    vid_batch = vid[-batch_size:]
                f = sess.run(feature, feed_dict={videos: video_batch})
                features += f[-actual_batch_size:],
    return np.concatenate(features, 0)

def compute_evals(real_act, fake_act):
    real_mu  = real_act.mean(0)
    real_sig = np.cov(real_act, rowvar=False)
    fake_mu  = fake_act.mean(0)
    fake_sig = np.cov(fake_act, rowvar=False)

    covmean  = sqrtm(real_sig @ fake_sig)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = np.sum((real_mu - fake_mu)**2) + np.trace(real_sig + fake_sig - 2 * covmean)

    kid = tfgan.eval.kernel_classifier_distance_from_activations(
            tf.constant(real_act), tf.constant(fake_act), max_block_size=1024)
    # return FID/FVD, KID/KVD
    return fid, kid.numpy()

In [34]:
# example for evaluation 
# Number of videos must be divisible by 16.
batch_size = 16
VIDEO_LENGTH = 16
height = 64
width = 64
channels = 3
# given some dummy samples that has shape 
# [batch_size, VIDEO_LENGTH, height, width, channels] in range 0-255
fake_samples  = np.random.random((batch_size, VIDEO_LENGTH, height, width, channels)) * 255
# dummy real data
real_samples  = np.random.random((batch_size, VIDEO_LENGTH, height, width, channels)) * 255
# compute features for both 
real_features = get_vid_features(real_samples)
fake_features = get_vid_features(fake_samples)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [35]:
#compute fvd and kvd from features generated from 3Dconv nets
fvd, kvd = compute_evals(real_features, fake_features)
print("FVD is", fvd)
print("KVD is", kvd)

FVD is 213.3925182341734
KVD is -6.2963867


In [36]:
# compute the features from inception net
real_incep_features = get_im_features(real_samples, batch_size=100)
fake_incep_features = get_im_features(fake_samples, batch_size=100)

In [38]:
#compute fvd and kvd from features generated from 3Dconv nets
fid, kid = compute_evals(real_incep_features, fake_incep_features)
print("FID is", fid)
print("KID is", kid )

FID is 3.49879263305247
KID is -1.7762184e-05
